In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def get_base_filepath():
    '''
    Access the filepath for the base folder of the project
    
    Input: None
    
    Output: The filepath to the root of the folder
    '''
    # Get current directory
    os.path.abspath(os.curdir)

    # Go up a directory level
    os.chdir('..')
    os.chdir('..')

    # Set baseline filepath to the project folder directory
    base_folder_filepath = os.path.abspath(os.curdir)
    return base_folder_filepath

In [3]:
def get_null_values(features, df):
    '''
    Generate a dataframe of the null value count and the minimum value
    
    Input:
        - A list of numeric features to find the null an min values for
        - A dataframe to access the features from
        
    Output: A dataframe of null value count and minimum value for each feature
    '''
    null_vals = dict()
    for col in features:
        null_vals[col] = (df[col].isnull().sum(), df[col].min())
        
    df_null_vals = pd.DataFrame(data=null_vals, index=['null_count', 'min_value'])
    return df_null_vals

In [4]:
# The folder for the project
base_folder_filepath = get_base_filepath() + 'Users/macbookair/Documents/Data/Phenotypic/'
phenotypics_filepath = base_folder_filepath + 'Sites/'
site_filepath = base_folder_filepath + 'Cleaned_Sites/'

In [5]:
# Create empty lists to store important values
features = [] # For the diagnosis

# Iterate through each file in the folder
for site_pheno in os.listdir(phenotypics_filepath):
    # Access the filepath to the phenotypic data
    site_pheno_filepath = os.path.join(phenotypics_filepath, site_pheno)
    
    # Check if the current item in the directory is a file
    if os.path.isfile(site_pheno_filepath):
        # Read the file as a dataframe
        df_pheno = pd.read_csv(site_pheno_filepath, index_col='ScanDir ID')
        
        # Add the diagnosis to the list
        features.append(df_pheno)

In [6]:
df_all_phenos = pd.concat(features, axis=0)
df_all_phenos

,Site,Gender,Age,Handedness,DX,Secondary Dx,ADHD Measure,ADHD Index,Inattentive,Hyper/Impulsive,...,QC_S1_Rest_1,QC_S1_Rest_2,QC_S1_Rest_3,QC_S1_Rest_4,QC_S1_Rest_5,QC_S1_Rest_6,QC_S1_Anat,QC_S2_Rest_1,QC_S2_Rest_2,QC_S2_Anat
ScanDir ID,,,,,,,,,,,,,,,,,,,,,
1000804,5,1.0,7.29,0.83,0,NaN,2.0,40.0,41.0,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1023964,5,1.0,8.29,0.57,3,NaN,2.0,60.0,56.0,48.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1057962,5,1.0,8.78,-999.00,1,NaN,2.0,77.0,81.0,80.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1099481,5,0.0,8.04,0.50,1,NaN,2.0,86.0,82.0,85.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1127915,5,0.0,12.44,0.21,0,NaN,2.0,42.0,43.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2950754,1,1.0,13.33,1.00,1,NaN,1.0,51.0,22.0,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2207418,1,1.0,13.17,1.00,1,SAD (previous),1.0,46.0,27.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2919220,1,1.0,13.58,1.00,3,NaN,1.0,45.0,28.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_all_phenos['DX'].isnull().sum()

0

In [8]:
#the quality of the fMRIs should not determine what diagnosis the patient has.
drop_features = ['QC_Rest_1', 'QC_Rest_2', 'QC_Rest_3', 'QC_Rest_4',
       'QC_Anatomical_1', 'QC_Anatomical_2', 'Study #', 'QC_S1_Rest_1',
       'QC_S1_Rest_2', 'QC_S1_Rest_3', 'QC_S1_Rest_4', 'QC_S1_Rest_5',
       'QC_S1_Rest_6', 'QC_S1_Anat', 'QC_S2_Rest_1', 'QC_S2_Rest_2',
       'QC_S2_Anat']

df_all_phenos_filtered = df_all_phenos.copy()
df_all_phenos_filtered = df_all_phenos_filtered.drop(drop_features, axis=1)
df_all_phenos_filtered

,Site,Gender,Age,Handedness,DX,Secondary Dx,ADHD Measure,ADHD Index,Inattentive,Hyper/Impulsive,IQ Measure,Verbal IQ,Performance IQ,Full2 IQ,Full4 IQ,Med Status
ScanDir ID,,,,,,,,,,,,,,,,
1000804,5,1.0,7.29,0.83,0,NaN,2.0,40.0,41.0,41.0,2,112.0,103.0,NaN,109,1.0
1023964,5,1.0,8.29,0.57,3,NaN,2.0,60.0,56.0,48.0,2,115.0,125.0,NaN,123,-999.0
1057962,5,1.0,8.78,-999.00,1,NaN,2.0,77.0,81.0,80.0,2,131.0,121.0,NaN,129,1.0
1099481,5,0.0,8.04,0.50,1,NaN,2.0,86.0,82.0,85.0,2,109.0,120.0,NaN,116,1.0
1127915,5,0.0,12.44,0.21,0,NaN,2.0,42.0,43.0,44.0,2,117.0,126.0,NaN,124,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2950754,1,1.0,13.33,1.00,1,NaN,1.0,51.0,22.0,29.0,3,137.0,109.0,NaN,127,1.0
2207418,1,1.0,13.17,1.00,1,SAD (previous),1.0,46.0,27.0,19.0,3,107.0,87.0,NaN,97,2.0
2919220,1,1.0,13.58,1.00,3,NaN,1.0,45.0,28.0,17.0,3,114.0,106.0,NaN,111,2.0


In [9]:
df_all_phenos_filtered.loc[df_all_phenos_filtered['Full4 IQ'].isnull(), 'Full4 IQ'] = df_all_phenos_filtered.loc[df_all_phenos_filtered['Full4 IQ'].isnull(), 'Full2 IQ']

In [10]:
df_all_phenos_filtered['IQ'] = df_all_phenos_filtered['Full4 IQ']
df_all_phenos_filtered = df_all_phenos_filtered.drop(['Full4 IQ', 'Full2 IQ'], axis=1)

In [11]:
df_all_phenos_filtered

,Site,Gender,Age,Handedness,DX,Secondary Dx,ADHD Measure,ADHD Index,Inattentive,Hyper/Impulsive,IQ Measure,Verbal IQ,Performance IQ,Med Status,IQ
ScanDir ID,,,,,,,,,,,,,,,
1000804,5,1.0,7.29,0.83,0,NaN,2.0,40.0,41.0,41.0,2,112.0,103.0,1.0,109
1023964,5,1.0,8.29,0.57,3,NaN,2.0,60.0,56.0,48.0,2,115.0,125.0,-999.0,123
1057962,5,1.0,8.78,-999.00,1,NaN,2.0,77.0,81.0,80.0,2,131.0,121.0,1.0,129
1099481,5,0.0,8.04,0.50,1,NaN,2.0,86.0,82.0,85.0,2,109.0,120.0,1.0,116
1127915,5,0.0,12.44,0.21,0,NaN,2.0,42.0,43.0,44.0,2,117.0,126.0,1.0,124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2950754,1,1.0,13.33,1.00,1,NaN,1.0,51.0,22.0,29.0,3,137.0,109.0,1.0,127
2207418,1,1.0,13.17,1.00,1,SAD (previous),1.0,46.0,27.0,19.0,3,107.0,87.0,2.0,97
2919220,1,1.0,13.58,1.00,3,NaN,1.0,45.0,28.0,17.0,3,114.0,106.0,2.0,111


In [13]:
df_all_phenos_filtered['IQ'].value_counts()

109    36
108    26
113    23
117    22
120    20
       ..
75      1
73      1
144     1
143     1
153     1
Name: IQ, Length: 70, dtype: int64

In [14]:
null_values = dict()
numeric_cols = ['Gender', 'Age', 'Handedness',
                'Verbal IQ', 'Performance IQ', 'IQ']

df_null_values_train = get_null_values(numeric_cols, df_all_phenos_filtered)

df_null_values_train.head()

,Gender,Age,Handedness,Verbal IQ,Performance IQ,IQ
null_count,1.0,0.00,1.0,140.0,140.0,0
min_value,0.0,7.09,-999.0,-999.0,-999.0,-999


In [15]:
#-999 is being used as a fill value for null values. It will be simpler to turn these into real null values and handle them with the other null values.
for col in df_all_phenos_filtered.columns:
    df_all_phenos_filtered.loc[df_all_phenos_filtered[col] == -999, col] = None

In [16]:
null_values = dict()
numeric_cols = ['Gender', 'Age', 'Handedness',
                'Verbal IQ', 'Performance IQ', 'IQ']

df_null_values_train = get_null_values(numeric_cols, df_all_phenos_filtered)

df_null_values_train.head()

,Gender,Age,Handedness,Verbal IQ,Performance IQ,IQ
null_count,1.0,0.00,7.0,194.0,194.0,12.0
min_value,0.0,7.09,-0.2,65.0,54.0,73.0


In [17]:
#filled with the most frequent value. The value 1 is the most frequent

df_all_phenos_filtered['Gender'] = df_all_phenos_filtered['Gender'].fillna(1)

In [18]:
# The most frequent value is 1 (right-handed), so this will be used to fill the missing values.

df_all_phenos_filtered['Handedness'] = df_all_phenos_filtered['Handedness'].fillna(1)

In [20]:
#measures handedness on a continuous scale unlike the other sites so make categorical values replace the continuous values.
df_all_phenos_filtered.loc[df_all_phenos_filtered['Handedness'] > 0, 'Handedness'] = 1
df_all_phenos_filtered.loc[df_all_phenos_filtered['Handedness'] < 0, 'Handedness'] = 0

In [22]:
#he null values for these features will be filled according to the average value for other subjects with the same diagnosis.
#Fill the nulll Verbal IQ values with the average value for each type of diagnosis.
df_all_phenos_filtered.groupby('DX')['Verbal IQ'].mean()

DX
0.0    114.563694
1.0    109.401639
2.0    105.666667
3.0    107.357895
Name: Verbal IQ, dtype: float64

In [23]:
df_all_phenos_filtered['Verbal IQ Filled'] = df_all_phenos_filtered['Verbal IQ']

df_all_phenos_filtered.loc[df_all_phenos_filtered['Verbal IQ Filled'].isnull(), 'Verbal IQ Filled'] = df_all_phenos_filtered.loc[df_all_phenos_filtered['Verbal IQ Filled'].isnull(), 'DX']

df_all_phenos_filtered.loc[df_all_phenos_filtered['Verbal IQ Filled'] == 0, 'Verbal IQ Filled'] = 114.563694
df_all_phenos_filtered.loc[df_all_phenos_filtered['Verbal IQ Filled'] == 1, 'Verbal IQ Filled'] = 109.401639
df_all_phenos_filtered.loc[df_all_phenos_filtered['Verbal IQ Filled'] == 2, 'Verbal IQ Filled'] = 105.666667
df_all_phenos_filtered.loc[df_all_phenos_filtered['Verbal IQ Filled'] == 3, 'Verbal IQ Filled'] = 107.357895

In [24]:
#Fill the nulll Performance IQ values with the average value for each type of diagnosis.
df_all_phenos_filtered.groupby('DX')['Performance IQ'].mean()

DX
0.0    109.984076
1.0    103.254098
2.0    113.333333
3.0    100.968421
Name: Performance IQ, dtype: float64

In [25]:
df_all_phenos_filtered['Performance IQ Filled'] = df_all_phenos_filtered['Performance IQ']

df_all_phenos_filtered.loc[df_all_phenos_filtered['Performance IQ Filled'].isnull(), 'Performance IQ Filled'] = df_all_phenos_filtered.loc[df_all_phenos_filtered['Performance IQ Filled'].isnull(), 'DX']

df_all_phenos_filtered.loc[df_all_phenos_filtered['Performance IQ Filled'] == 0, 'Performance IQ Filled'] = 109.984076
df_all_phenos_filtered.loc[df_all_phenos_filtered['Performance IQ Filled'] == 1, 'Performance IQ Filled'] = 103.254098
df_all_phenos_filtered.loc[df_all_phenos_filtered['Performance IQ Filled'] == 2, 'Performance IQ Filled'] = 113.333333
df_all_phenos_filtered.loc[df_all_phenos_filtered['Performance IQ Filled'] == 3, 'Performance IQ Filled'] = 100.968421

In [26]:
#Fill the nulll IQ values with the average value for each type of diagnosis.
df_all_phenos_filtered.groupby('DX')['IQ'].mean()

DX
0.0    113.745098
1.0    107.620690
2.0    110.800000
3.0    104.710280
Name: IQ, dtype: float64

In [28]:
df_all_phenos_filtered['IQ Filled'] = df_all_phenos_filtered['IQ']

df_all_phenos_filtered.loc[df_all_phenos_filtered['IQ Filled'].isnull(), 'IQ Filled'] = df_all_phenos_filtered.loc[df_all_phenos_filtered['IQ Filled'].isnull(), 'DX']

df_all_phenos_filtered.loc[df_all_phenos_filtered['IQ Filled'] == 0, 'IQ Filled'] = 113.745098
df_all_phenos_filtered.loc[df_all_phenos_filtered['IQ Filled'] == 1, 'IQ Filled'] = 107.620690
df_all_phenos_filtered.loc[df_all_phenos_filtered['IQ Filled'] == 2, 'IQ Filled'] = 110.800000
df_all_phenos_filtered.loc[df_all_phenos_filtered['IQ Filled'] == 3, 'IQ Filled'] = 104.710280

In [29]:
null_values = dict()
numeric_cols = ['Gender', 'Age', 'Handedness', 'Verbal IQ', 'Verbal IQ Filled',
                'Performance IQ', 'Performance IQ Filled', 'IQ', 'IQ Filled']

df_null_values_train = get_null_values(numeric_cols, df_all_phenos_filtered)

df_null_values_train.head()

,Gender,Age,Handedness,Verbal IQ,Verbal IQ Filled,Performance IQ,Performance IQ Filled,IQ,IQ Filled
null_count,0.0,0.00,0.0,194.0,0.0,194.0,0.0,12.0,0.0
min_value,0.0,7.09,0.0,65.0,65.0,54.0,54.0,73.0,73.0


In [30]:
features = ['Gender', 'Age', 'Handedness', 'Verbal IQ', 'Verbal IQ Filled',
            'Performance IQ', 'Performance IQ Filled', 'IQ', 'IQ Filled', 'DX']

df_all_phenos_filtered[features]

,Gender,Age,Handedness,Verbal IQ,Verbal IQ Filled,Performance IQ,Performance IQ Filled,IQ,IQ Filled,DX
ScanDir ID,,,,,,,,,,
1000804,1.0,7.29,1.0,112.0,112.0,103.0,103.0,109.0,109.0,0.0
1023964,1.0,8.29,1.0,115.0,115.0,125.0,125.0,123.0,123.0,3.0
1057962,1.0,8.78,1.0,131.0,131.0,121.0,121.0,129.0,129.0,1.0
1099481,0.0,8.04,1.0,109.0,109.0,120.0,120.0,116.0,116.0,1.0
1127915,0.0,12.44,1.0,117.0,117.0,126.0,126.0,124.0,124.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2950754,1.0,13.33,1.0,137.0,137.0,109.0,109.0,127.0,127.0,1.0
2207418,1.0,13.17,1.0,107.0,107.0,87.0,87.0,97.0,97.0,1.0
2919220,1.0,13.58,1.0,114.0,114.0,106.0,106.0,111.0,111.0,3.0


In [31]:
df_all_phenos_filtered[features].to_csv(base_folder_filepath + 'offmind2023.7.13-Cleaned_Phenotypic_All_Subjects.csv')

In [32]:
site_names = ['KKI', 'NYU', 'OHSU', 'Peking', 'Pittsburgh', 'WashU']
site_nums = [3, 5, 6, 1, 7, 8]

In [33]:
# For each index in the site numbers list...
for i in range(len(site_nums)):
    # Locate the subjects from that site and store them as a dataframe
    site_pheno = df_all_phenos_filtered.loc[df_all_phenos_filtered['Site'] == site_nums[i]]
    
    # Save the dataframe with select features in the correct folder
    site_pheno[features].to_csv(site_filepath + 'offmind2023.7.13-Cleaned_' + site_names[i] + '_pheno.csv')